In [1]:
import sys
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from copy import deepcopy

%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
np.__version__

'1.19.2'

In [3]:
import das.pytorch.estimator as pe
import torch
torch.__version__

'1.3.1'

In [4]:
torch.cuda.is_available()

True

In [5]:
sys.path.insert(0,"/media/cfs/wenhao71/workspace/torch_ecg/")
from torch_ecg.train.train_seq_lab_net_cpsc2019.cfg import TrainCfg, ModelCfg
from torch_ecg.train.train_seq_lab_net_cpsc2019.train import train, evaluate
from torch_ecg.train.train_seq_lab_net_cpsc2019.model import ECG_SEQ_LAB_NET_CPSC2019
from torch_ecg.train.train_seq_lab_net_cpsc2019.utils import init_logger, dict_to_str


2020-11-12 09:25:48,522 (WARNING) __init__.py:482] Matplotlib created a temporary config/cache directory at /tmp/matplotlib-r42n02f0 because the default path (/work/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
2020-11-12 09:25:48,523 (INFO) font_manager.py:1423] Generating new fontManager, this may take some time...


In [6]:
train_config = deepcopy(TrainCfg)

In [7]:
train_config.model_name = "cnn"

In [ ]:
# train_config.model_name = "crnn"

In [8]:
ModelCfg

{'skip_dist': 250.0,
 'torch_dtype': 'float',
 'fs': 500,
 'spacing': 2.0,
 'classes': ['N'],
 'n_leads': 1,
 'seq_lab_crnn': {'fs': 500,
  'classes': ['N'],
  'n_leads': 1,
  'skip_dist': 250.0,
  'torch_dtype': 'float',
  'cnn': {'name': 'multi_scopic',
   'multi_scopic': {'groups': 1,
    'scopes': [[[1], [1, 1], [1, 1, 1]],
     [[2], [2, 4], [8, 8, 8]],
     [[4], [4, 8], [16, 32, 64]]],
    'filter_lengths': [[11, 7, 5], [11, 7, 5], [11, 7, 5]],
    'subsample_lengths': [2, 2, 2],
    'num_filters': [[32, 64, 128], [32, 64, 128], [32, 64, 128]],
    'dropouts': [[0, 0.2, 0], [0, 0.2, 0], [0, 0.2, 0]],
    'bias': True,
    'kernel_initializer': 'he_normal',
    'kw_initializer': {},
    'activation': 'relu',
    'kw_activation': {'inplace': True},
    'block': {'subsample_mode': 'max',
     'bias': True,
     'kernel_initializer': 'he_normal',
     'kw_initializer': {},
     'activation': 'relu',
     'kw_activation': {'inplace': True}}}},
  'rnn': {'name': 'lstm',
   'lstm': {'h

In [9]:
ModelCfg[f"seq_lab_{train_config.model_name.lower()}"]

{'fs': 500,
 'classes': ['N'],
 'n_leads': 1,
 'skip_dist': 250.0,
 'torch_dtype': 'float',
 'cnn': {'name': 'multi_scopic',
  'multi_scopic': {'groups': 1,
   'scopes': [[[1], [1, 1], [1, 1, 1]],
    [[2], [2, 4], [8, 8, 8]],
    [[4], [4, 8], [16, 32, 64]]],
   'filter_lengths': [[11, 7, 5], [11, 7, 5], [11, 7, 5]],
   'subsample_lengths': [2, 2, 2],
   'num_filters': [[32, 64, 128], [32, 64, 128], [32, 64, 128]],
   'dropouts': [[0, 0.2, 0], [0, 0.2, 0], [0, 0.2, 0]],
   'bias': True,
   'kernel_initializer': 'he_normal',
   'kw_initializer': {},
   'activation': 'relu',
   'kw_activation': {'inplace': True},
   'block': {'subsample_mode': 'max',
    'bias': True,
    'kernel_initializer': 'he_normal',
    'kw_initializer': {},
    'activation': 'relu',
    'kw_activation': {'inplace': True}}}},
 'rnn': {'name': 'none'},
 'attn': {'name': 'se',
  'se': {'reduction': 8,
   'activation': 'relu',
   'kw_activation': {'inplace': True},
   'bias': True,
   'kernel_initializer': 'he_norma

In [10]:
model_config = deepcopy(ModelCfg[f"seq_lab_{train_config.model_name.lower()}"])

In [11]:
train_config

{'fs': 500,
 'db_dir': '/media/cfs/wenhao71/data/CPSC2019/train/',
 'log_dir': '/media/cfs/wenhao71/workspace/torch_ecg/torch_ecg/train/train_seq_lab_net_cpsc2019/log',
 'checkpoints': '/media/cfs/wenhao71/workspace/torch_ecg/torch_ecg/train/train_seq_lab_net_cpsc2019/checkpoints',
 'keep_checkpoint_max': 50,
 'train_ratio': 0.8,
 'input_len': 5000,
 'classes': ['N'],
 'n_leads': 1,
 'bias_thr': 37.5,
 'skip_dist': 250.0,
 'label_smoothing': 0.0,
 'random_normalize': True,
 'random_normalize_mean': [-0.05, 0.1],
 'random_normalize_std': [0.08, 0.32],
 'baseline_wander': True,
 'bw': True,
 'bw_fs': array([0.33, 0.1 , 0.05, 0.01]),
 'bw_ampl_ratio': array([[0.01, 0.01, 0.02, 0.03],
        [0.01, 0.02, 0.04, 0.05],
        [0.1 , 0.06, 0.04, 0.02],
        [0.02, 0.04, 0.07, 0.1 ],
        [0.05, 0.1 , 0.16, 0.25],
        [0.1 , 0.15, 0.25, 0.3 ],
        [0.25, 0.25, 0.3 , 0.35]]),
 'bw_gaussian': array([[0.   , 0.   ],
        [0.   , 0.   ],
        [0.   , 0.   ],
        [0.   , 0

In [12]:
model = ECG_SEQ_LAB_NET_CPSC2019(
    n_leads=train_config.n_leads,
    input_len=train_config.input_len,
    config=model_config,
)

classes (totally 1) for prediction:['N']
configuration of ECG_SEQ_LAB_NET_CPSC2019 is as follows
{
    "cnn": {
        "name": "multi_scopic",
        "multi_scopic": {
            "groups": 1,
            "scopes": [
                [
                    [
                        1
                    ],
                    [
                        1, 1
                    ],
                    [
                        1, 1, 1
                    ]
                ],
                [
                    [
                        2
                    ],
                    [
                        2, 4
                    ],
                    [
                        8, 8, 8
                    ]
                ],
                [
                    [
                        4
                    ],
                    [
                        4, 8
                    ],
                    [
                        16, 32, 64
                    ]
                ]
     

In [13]:
torch.cuda.device_count()

1

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [15]:
model.to(device=device)

ECG_SEQ_LAB_NET_CPSC2019(
  (cnn): MultiScopicCNN(
    (branches): ModuleDict(
      (branch_0): MultiScopicBranch(
        (block_0): MultiScopicBasicBlock(
          (ca_0): Conv_Bn_Activation(
            (conv1d): Conv1d(1, 32, kernel_size=(11,), stride=(1,), padding=(5,))
            (activation_relu): ReLU(inplace=True)
          )
          (bn): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (down): DownSample(
            (down_sample): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
          )
        )
        (block_1): MultiScopicBasicBlock(
          (ca_0): Conv_Bn_Activation(
            (conv1d): Conv1d(32, 64, kernel_size=(7,), stride=(1,), padding=(3,))
            (activation_relu): ReLU(inplace=True)
          )
          (ca_1): Conv_Bn_Activation(
            (conv1d): Conv1d(64, 64, kernel_size=(7,), stride=(1,), padding=(3,))
            (activation_relu): ReLU(inplace=True)
          )
    

In [16]:
logger = init_logger(log_dir=train_config.log_dir)
logger.info(f"\n{'*'*20}   Start Training   {'*'*20}\n")
logger.info(f'Using device {device}')
logger.info(f"Using torch of version {torch.__version__}")
logger.info(f'with configuration\n{dict_to_str(train_config)}')

log file path: /media/cfs/wenhao71/workspace/torch_ecg/torch_ecg/train/train_seq_lab_net_cpsc2019/log/log_2020-11-12-09-25-52.txt
levels of c_handler and f_handler are set WARNING


In [17]:
hehe_input = torch.rand((2,1,4000)).to(device,dtype=torch.float32)

In [18]:
pred, rpeaks = model.inference(hehe_input,0.5)

In [19]:
rpeaks

[array([ 336,  508,  648,  824, 1108, 1236, 1504, 1660, 1920, 2156, 2684,
        2932, 3184, 3336, 3588, 3688]),
 array([ 384,  500,  616,  908, 1156, 1388, 1592, 1728, 1928, 2116, 2288,
        2440, 2656, 2764, 3088, 3280, 3456, 3624])]

In [21]:
train_config.debug = True
train_config.cnn_name = model_config.cnn.name
train_config.rnn_name = model_config.rnn.name
train_config.attn_name = model_config.attn.name
train_config

{'fs': 500,
 'db_dir': '/media/cfs/wenhao71/data/CPSC2019/train/',
 'log_dir': '/media/cfs/wenhao71/workspace/torch_ecg/torch_ecg/train/train_seq_lab_net_cpsc2019/log',
 'checkpoints': '/media/cfs/wenhao71/workspace/torch_ecg/torch_ecg/train/train_seq_lab_net_cpsc2019/checkpoints',
 'keep_checkpoint_max': 50,
 'train_ratio': 0.8,
 'input_len': 5000,
 'classes': ['N'],
 'n_leads': 1,
 'bias_thr': 37.5,
 'skip_dist': 250.0,
 'label_smoothing': 0.0,
 'random_normalize': True,
 'random_normalize_mean': [-0.05, 0.1],
 'random_normalize_std': [0.08, 0.32],
 'baseline_wander': True,
 'bw': True,
 'bw_fs': array([0.33, 0.1 , 0.05, 0.01]),
 'bw_ampl_ratio': array([[0.01, 0.01, 0.02, 0.03],
        [0.01, 0.02, 0.04, 0.05],
        [0.1 , 0.06, 0.04, 0.02],
        [0.02, 0.04, 0.07, 0.1 ],
        [0.05, 0.1 , 0.16, 0.25],
        [0.1 , 0.15, 0.25, 0.3 ],
        [0.25, 0.25, 0.3 , 0.35]]),
 'bw_gaussian': array([[0.   , 0.   ],
        [0.   , 0.   ],
        [0.   , 0.   ],
        [0.   , 0

In [22]:
train(
    model=model,
    config=train_config,
    device=device,
    logger=logger,
    debug=train_config.debug,
)

training configurations are as follows:
{
    "fs": 500
    "db_dir": "/media/cfs/wenhao71/data/CPSC2019/train/"
    "log_dir": "/media/cfs/wenhao71/workspace/torch_ecg/torch_ecg/train/train_seq_lab_net_cpsc2019/log"
    "checkpoints": "/media/cfs/wenhao71/workspace/torch_ecg/torch_ecg/train/train_seq_lab_net_cpsc2019/checkpoints"
    "keep_checkpoint_max": 50
    "train_ratio": 0.8
    "input_len": 5000
    "classes": [
        "N"
    ]
    "n_leads": 1
    "bias_thr": 37.5
    "skip_dist": 250.0
    "label_smoothing": 0.0
    "random_normalize": True
    "random_normalize_mean": [
        -0.05, 0.1
    ]
    "random_normalize_std": [
        0.08, 0.32
    ]
    "baseline_wander": True
    "bw": True
    "bw_fs": [0.33 0.1  0.05 0.01]
    "bw_ampl_ratio": [[0.01 0.01 0.02 0.03]
 [0.01 0.02 0.04 0.05]
 [0.1  0.06 0.04 0.02]
 [0.02 0.04 0.07 0.1 ]
 [0.05 0.1  0.16 0.25]
 [0.1  0.15 0.25 0.3 ]
 [0.25 0.25 0.3  0.35]]
    "bw_gaussian": [[0.    0.   ]
 [0.    0.   ]
 [0.    0.   ]
 [0.

QRS_acc: 0.0926
Scoring complete.
QRS_acc: 0.0758
Scoring complete.

                Train epoch_1:
                --------------------
                train/epoch_loss:        4.917660117149353
                train/qrs_score:         0.0926
            
                test/qrs_score:          0.0758
                ---------------------------------
            



QRS_acc: 0.1437
Scoring complete.
QRS_acc: 0.1268
Scoring complete.

                Train epoch_2:
                --------------------
                train/epoch_loss:        3.5938060581684113
                train/qrs_score:         0.1437
            
                test/qrs_score:          0.1268
                ---------------------------------
            



Train step_20: loss : 0.40978461503982544
QRS_acc: 0.1999
Scoring complete.
QRS_acc: 0.1767
Scoring complete.

                Train epoch_3:
                --------------------
                train/epoch_loss:        2.969119608402252
                train/qrs_score:         0.1999
            
                test/qrs_score:          0.1767
                ---------------------------------
            



QRS_acc: 0.2921
Scoring complete.
QRS_acc: 0.2535
Scoring complete.

                Train epoch_4:
                --------------------
                train/epoch_loss:        2.58990678191185
                train/qrs_score:         0.2921
            
                test/qrs_score:          0.2535
                ---------------------------------
            



...

QRS_acc: 0.9226
Scoring complete.
QRS_acc: 0.8795
Scoring complete.

                Train epoch_296:
                --------------------
                train/epoch_loss:        0.44392411038279533
                train/qrs_score:         0.9226
            
                test/qrs_score:          0.8795
                ---------------------------------
            



QRS_acc: 0.9241
Scoring complete.
QRS_acc: 0.8785
Scoring complete.

                Train epoch_297:
                --------------------
                train/epoch_loss:        0.42752864211797714
                train/qrs_score:         0.9241
            
                test/qrs_score:          0.8785
                ---------------------------------
            



Train step_2080: loss : 0.05741260200738907
QRS_acc: 0.9214
Scoring complete.
QRS_acc: 0.878
Scoring complete.

                Train epoch_298:
                --------------------
                train/epoch_loss:        0.4364863410592079
                train/qrs_score:         0.9214
            
                test/qrs_score:          0.878
                ---------------------------------
            



QRS_acc: 0.9204
Scoring complete.
QRS_acc: 0.8798
Scoring complete.

                Train epoch_299:
                --------------------
                train/epoch_loss:        0.42523564770817757
                train/qrs_score:         0.9204
            
                test/qrs_score:          0.8798
                ---------------------------------
            



Train step_2100: loss : 0.061116691678762436
QRS_acc: 0.9181
Scoring complete.
QRS_acc: 0.8772
Scoring complete.

                Train epoch_300:
                --------------------
                train/epoch_loss:        0.43124497309327126
                train/qrs_score:         0.9181
            
                test/qrs_score:          0.8772
                ---------------------------------
            



In [ ]:
# job = pe.PyTorch(
#     entry_point='/media/cfs/wenhao71/workspace/torch_ecg/torch_ecg/train/train_seq_lab_net_cpsc2019/train.py',
#     framework_version='1.3.1',
#     train_gpu_count=1,
#     hyperparameters = {'debug': 'True', 'batch-size':256},
# #     hyperparameters = {'model-name':model_name, 'rnn-name':rnn, 'debug': 'True', 'batch-size':bs},
# )

In [ ]:
# job.fit(base_job_name=f'cpsc2019-seq-lab')